## Downloading the Data + PIP Installs

Data is already downloaded and provided in the repository, but it needs to be unzipped.

In [ ]:
import platform
import sys
mysystem = platform.system()

if mysystem != 'Windows':
    !unzip ../../Data/hotelReservations.zip
    !{sys.executable} -m pip install pandas --user
    !{sys.executable} -m pip install numpy --user
    !{sys.executable} -m pip install seaborn --user
    !{sys.executable} -m pip install sklearn --user
    !{sys.executable} -m pip install tensorflow --user
else:
    print('For windows, manually unzip the data and install pandas, numpy, seaborn, sklearn, and tensorflow')

## Imports 

Here we include import statements and needed libraries to perform this assignment.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

## Finishing Up Preprocessing and Building our First Model

#### Preprocessing

First, transform our data and remove the columns that we will not or cannot use for our model.

We will remove the columns: country, agent, and company. This is due to being categorical attributes with missing data values as it would be inaccurate to impute the data with a set value.

Another two columns we will remove is reservation_status_date and reservation_status due to other columns being able to provide essential information for the model. We only need to know if a reservation is cancelled or not, which is provided already.

In [4]:
df = pd.read_csv('./hotel_bookings.csv')
df = df.drop(columns=['country', 'agent', 'company', 'reservation_status_date', 'reservation_status'])
display(df)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,0,C,C,3,No Deposit,0,Transient,0.00,0,0
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,0,C,C,4,No Deposit,0,Transient,0.00,0,0
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,0,A,C,0,No Deposit,0,Transient,75.00,0,0
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,0,A,A,0,No Deposit,0,Transient,75.00,0,0
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,A,A,0,No Deposit,0,Transient,98.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,0,A,A,0,No Deposit,0,Transient,96.14,0,0
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,0,E,E,0,No Deposit,0,Transient,225.43,0,2
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,0,D,D,0,No Deposit,0,Transient,157.71,0,4
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,0,A,A,0,No Deposit,0,Transient,104.40,0,0


Next, we need to impute our data.

The only column with missing values is the children's column. Thus, for values that are NULL, we will rewrite their value to zero.

In [5]:
df.isna().any()

hotel                             False
is_canceled                       False
lead_time                         False
arrival_date_year                 False
arrival_date_month                False
arrival_date_week_number          False
arrival_date_day_of_month         False
stays_in_weekend_nights           False
stays_in_week_nights              False
adults                            False
children                           True
babies                            False
meal                              False
market_segment                    False
distribution_channel              False
is_repeated_guest                 False
previous_cancellations            False
previous_bookings_not_canceled    False
reserved_room_type                False
assigned_room_type                False
booking_changes                   False
deposit_type                      False
days_in_waiting_list              False
customer_type                     False
adr                               False


In [6]:
df = df.replace(np.nan, 0)
df['children'].value_counts()

0.0     110800
1.0       4861
2.0       3652
3.0         76
10.0         1
Name: children, dtype: int64

Next, we need to scale our data. 

To do so, we need to determine the numerical attributes in our dataframe. From our data distribution exploration, the numerical attributes are: lead_time, stays_in_week_nights, stays_in_weekend_nights, adults, children, babies, previous_cancellations, previous_bookings_not_canceled, booking_changes, days_in_waiting_list, adr, and total_of_special_requests.

In [7]:
from sklearn.preprocessing import MinMaxScaler

norm_scaler = MinMaxScaler()
numericalAttribs = ['lead_time', 
                    'stays_in_week_nights', 
                    'stays_in_weekend_nights', 
                    'adults',
                    'children',
                    'babies',
                    'previous_cancellations',
                    'previous_bookings_not_canceled',
                    'booking_changes',
                    'days_in_waiting_list',
                    'adr',
                    'total_of_special_requests']

df[numericalAttribs] = norm_scaler.fit_transform(df[numericalAttribs])
display(df[numericalAttribs])

,lead_time,stays_in_week_nights,stays_in_weekend_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,total_of_special_requests
0,0.464043,0.00,0.000000,0.036364,0.0,0.0,0.0,0.0,0.142857,0.0,0.001180,0.0
1,1.000000,0.00,0.000000,0.036364,0.0,0.0,0.0,0.0,0.190476,0.0,0.001180,0.0
2,0.009498,0.02,0.000000,0.018182,0.0,0.0,0.0,0.0,0.000000,0.0,0.015053,0.0
3,0.017639,0.02,0.000000,0.018182,0.0,0.0,0.0,0.0,0.000000,0.0,0.015053,0.0
4,0.018996,0.04,0.000000,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.019307,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.031208,0.10,0.105263,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.018963,0.0
119386,0.138399,0.10,0.105263,0.054545,0.0,0.0,0.0,0.0,0.000000,0.0,0.042877,0.4
119387,0.046133,0.10,0.105263,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.030351,0.8
119388,0.147897,0.10,0.105263,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.020491,0.0


Next, we are going to encode our categorical attributes in the dataframe.

To do so, we need to determine our categorical attributes in our dataframe. From our data exploration, the categorical attributes are: hotel, arrival_date_month, arrival_date_year, arrival_date_week_number, arrival_date_day_of_month, meal, market_segment, distribution_channel, is_repeated_guest, reserved_room_type, assigned_room_type, deposit_type, customer_type, and required_car_parking_spaces.

In [8]:
categoricalAttribs = ['hotel',
                      'arrival_date_month',
                      'arrival_date_year',
                      'arrival_date_week_number',
                      'arrival_date_day_of_month',
                      'meal',
                      'market_segment',
                      'distribution_channel',
                      'is_repeated_guest',
                      'reserved_room_type',
                      'assigned_room_type',
                      'deposit_type',
                      'customer_type',
                      'required_car_parking_spaces']

encodings = {}
for a in categoricalAttribs:
    keys = {}
    iter = 0
    for val in list(df[a].unique()):
        keys[val] = iter
        iter += 1
    encodings[a] = keys

df = df.replace(encodings)
df[categoricalAttribs]

,hotel,arrival_date_month,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,meal,market_segment,distribution_channel,is_repeated_guest,reserved_room_type,assigned_room_type,deposit_type,customer_type,required_car_parking_spaces
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,1,0,1,1,0,0,0
4,0,0,0,0,0,0,2,2,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,1,1,2,8,29,0,3,2,0,1,1,0,0,0
119386,1,1,2,8,30,0,2,2,0,3,3,0,0,0
119387,1,1,2,8,30,0,2,2,0,2,2,0,0,0
119388,1,1,2,8,30,0,2,2,0,1,1,0,0,0


#### Building Model

First, lets build and train our first model by splitting the data into X and y | Test and Train for cross validation. 

In [13]:
from sklearn.model_selection import train_test_split 
Train, Test = train_test_split(df, test_size=0.3)

X_train = Train.drop('is_canceled', axis = 1)
X_test = Test.drop('is_canceled', axis = 1)

y_train = Train['is_canceled']
y_test = Test['is_canceled']

display(X_train, X_test, y_train, y_test)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
104012,1,0.016282,2,6,27,1,0.052632,0.02,0.018182,0.0,...,0.000000,2,2,0.000000,0,0.000000,0,0.025596,0,0.2
104730,1,0.094980,2,6,29,14,0.105263,0.08,0.036364,0.0,...,0.000000,1,1,0.000000,0,0.000000,0,0.018493,0,0.4
29463,0,0.036635,1,3,17,27,0.000000,0.02,0.036364,0.0,...,0.000000,1,3,0.047619,0,0.000000,0,0.018752,0,0.4
80304,1,0.044776,0,5,23,8,0.000000,0.04,0.036364,0.0,...,0.000000,1,1,0.000000,2,0.000000,0,0.021526,0,0.0
98163,1,0.024423,1,2,12,21,0.052632,0.06,0.036364,0.0,...,0.000000,1,1,0.047619,0,0.000000,0,0.029018,0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,0,0.000000,0,3,15,11,0.000000,0.00,0.036364,0.0,...,0.000000,2,1,0.000000,0,0.000000,0,0.001180,0,0.4
16122,0,0.122117,0,1,6,10,0.000000,0.10,0.036364,0.1,...,0.000000,3,3,0.047619,0,0.000000,0,0.034406,1,0.4
104624,1,0.023066,2,6,28,12,0.105263,0.08,0.036364,0.2,...,0.000000,5,4,0.000000,0,0.000000,0,0.028000,0,0.2
45292,1,0.093623,0,3,17,24,0.105263,0.02,0.036364,0.0,...,0.000000,1,1,0.000000,0,0.148338,2,0.017026,0,0.0


,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
73244,1,0.141113,2,1,6,16,0.000000,0.06,0.036364,0.0,...,0.0,2,2,0.095238,0,0.000000,0,0.029850,0,0.2
46053,1,0.268657,0,5,22,4,0.105263,0.02,0.036364,0.0,...,0.0,1,1,0.000000,2,0.000000,0,0.017827,0,0.0
87573,1,0.093623,1,9,43,17,0.052632,0.06,0.036364,0.0,...,0.0,2,2,0.000000,0,0.000000,2,0.021541,0,0.2
42815,1,0.006784,0,2,10,9,0.000000,0.04,0.036364,0.0,...,0.0,1,1,0.000000,0,0.000000,0,0.024116,0,0.2
30706,0,0.008141,1,4,21,21,0.105263,0.16,0.036364,0.0,...,0.0,1,1,0.000000,0,0.000000,0,0.007837,0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97800,1,0.036635,1,2,12,19,0.000000,0.02,0.054545,0.0,...,0.0,1,1,0.095238,0,0.000000,2,0.024301,0,0.0
102677,1,0.039349,1,5,22,1,0.000000,0.04,0.036364,0.0,...,0.0,1,1,0.000000,0,0.000000,2,0.016902,0,0.0
46358,1,0.217096,0,5,26,29,0.000000,0.06,0.036364,0.0,...,0.0,1,1,0.000000,0,0.283887,2,0.014252,0,0.0
48070,1,0.024423,1,8,38,14,0.000000,0.10,0.036364,0.0,...,0.0,3,3,0.000000,0,0.000000,0,0.025004,0,0.0


104012    0
104730    0
29463     0
80304     1
98163     0
         ..
2408      0
16122     0
104624    0
45292     0
80793     1
Name: is_canceled, Length: 83573, dtype: int64

73244     1
46053     1
87573     0
42815     1
30706     0
         ..
97800     0
102677    0
46358     0
48070     1
38188     0
Name: is_canceled, Length: 35817, dtype: int64

Next, we decide which model to use for predicting our yhat values. These are the prediction values for whether a hotel booking will be canceled. Additionally, it will help us determine the error of our Training and Testing data

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.naive_bayes import CategoricalNB
from keras.models import Sequential
from keras.layers import Dense
#SVM?
#KNN?
#Logistic Regression?
#DTL?

For our first model, we will go with a simple Neural Network model with a few dense layers.

In [14]:
model = Sequential() 

model.add(Dense(units = 50, activation = 'relu', input_dim = 26))
model.add(Dense(units = 35, activation = 'relu'))
model.add(Dense(units = 20, activation = 'tanh'))
model.add(Dense(units = 10, activation = 'linear'))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')
model.fit(X_train.astype('float'), y_train, batch_size = 1, epochs = 10)

Epoch 1/10
83573/83573 [==============================] - 43s 513us/step - loss: 0.5329
Epoch 2/10
83573/83573 [==============================] - 42s 508us/step - loss: 0.5238
Epoch 3/10
83573/83573 [==============================] - 42s 506us/step - loss: 0.5278
Epoch 4/10
83573/83573 [==============================] - 42s 505us/step - loss: 0.5320
Epoch 5/10
83573/83573 [==============================] - 42s 508us/step - loss: 0.5340
Epoch 6/10
83573/83573 [==============================] - 42s 505us/step - loss: 0.5386
Epoch 7/10
83573/83573 [==============================] - 43s 513us/step - loss: 0.5412
Epoch 8/10
83573/83573 [==============================] - 43s 509us/step - loss: 0.5391
Epoch 9/10
83573/83573 [==============================] - 43s 512us/step - loss: 0.5317
Epoch 10/10
83573/83573 [==============================] - 43s 515us/step - loss: 0.5341


Let us evaluate the model comparing training versus testing error.

In [16]:
from sklearn.metrics import classification_report

yhat_train_model = model.predict(X_train.astype(float))
yhat_train_model = [1 if y>=0.5 else 0 for y in yhat_train_model]
yhat_test_model = model.predict(X_test.astype(float))
yhat_test_model = [1 if y>=0.5 else 0 for y in yhat_test_model]

print(classification_report(y_train, yhat_train_model))
print(classification_report(y_test, yhat_test_model))

1120/1120 [==============================] - 0s 436us/step
              precision    recall  f1-score   support

           0       0.78      0.91      0.84     52506
           1       0.80      0.57      0.67     31067

    accuracy                           0.79     83573
   macro avg       0.79      0.74      0.75     83573
weighted avg       0.79      0.79      0.78     83573

              precision    recall  f1-score   support

           0       0.79      0.91      0.85     22660
           1       0.80      0.59      0.67     13157

    accuracy                           0.79     35817
   macro avg       0.79      0.75      0.76     35817
weighted avg       0.79      0.79      0.78     35817



#### Model fitting

The model does not seem to be overfitting, because although the training was 78% accurate, the testing was 78% accurate as well. Therefore, we do not have a significantly more accurate model that overfits the data when it moves from train to test data.

However, the model does seem to be underfitting, because the train and test data was 78%. Thus, the fitting for this model is underfitting as its not fully predicting if a reservation will be cancelled with high accuracy.